# Tokenize and save data

In [1]:
%load_ext music21.ipython21

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.text import *

In [4]:
import pandas as pd
from fastai_data import *

In [8]:
from encode_data import *

In [9]:
from midi_data import *

In [10]:
from midi_transform import *

In [11]:
import shutil

## Retrieve File

In [12]:
# source_dir = 'midi_transcribe_v1_simple'
path = Path(f'data/midi/')
csv_path = path/'metadata/midi_sources.csv'
# path.ls()[:5]

In [13]:
csv = pd.read_csv(csv_path)
# csv = csv.loc[csv[source_dir].notna()];

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,14,15,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
file = csv[csv.source == 'hooktheory'].iloc[200]; file

Unnamed: 0                                                         200
index                                                              200
artist                                         wolfgang-amadeus-mozart
bpm                                                                194
genres                                                       Classical
ht_bpm                                                             194
ht_key                                                              Bb
ht_mode                                                              1
ht_offset                                                            2
ht_time_signature                                                    4
inferred_key                                                   F major
inferred_offset                                                     -5
instruments                                                Piano,Piano
midi                 data/midi/midi_sources/hooktheory/pianoroll/w/...
midi_t

In [15]:
file = file.midi

### Copy and play

In [16]:
copy_file = '/tmp/test_encoding.MID'
shutil.copy(file, copy_file)

'/tmp/test_encoding.MID'

In [17]:
original_stream = file2stream(copy_file)
original_stream.show('midi')

In [18]:
# play_midi(copy_file)

In [19]:
for n in list(original_stream.flat[:200]):
    if isinstance(n, (music21.note.Note, music21.chord.Chord)):
        print(n.offset, n.volume, n.duration, n.duration.quarterLength, n)

0.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 4.0> 4.0 <music21.note.Note B->
0.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 4.0> 4.0 <music21.chord.Chord B-2 D3 F3>
4.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 3.0> 3.0 <music21.note.Note F>
4.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 4.0> 4.0 <music21.chord.Chord A2 C3 F3>
7.5 <music21.volume.Volume realized=0.79> <music21.duration.Duration 0.5> 0.5 <music21.note.Note F>
8.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 1.0> 1.0 <music21.note.Note E->
8.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 4.0> 4.0 <music21.chord.Chord G2 B-2 E-3>
9.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 1.0> 1.0 <music21.note.Note G>
10.0 <music21.volume.Volume realized=0.79> <music21.duration.Duration 1.0> 1.0 <music21.note.Note B->
12.0 <music21.volume.Volume realized=0.79> <music21.duration.Durat

### Transpose test 2

In [20]:
mfonly_file = '/tmp/mfonly.MID'
music_file = compress_midi_file(copy_file, cutoff=6) # remove non note tracks and standardize instruments
music_file = transpose_midi_file(music_file, 10) # transpose to keyc if offset passed
music_file.open(mfonly_file, attrib='wb')
music_file.write()
music_file.close()

### Encode2text

In [21]:
transpose_file = '/tmp/test_transpose.MID'
transform_midi(copy_file, transpose_file)

Inferring offset


'/tmp/test_transpose.MID'

In [22]:
text = midi2str(transpose_file)
text_stream = str2stream(text)
text_stream.show('text')

{0.0} <music21.stream.Part 0x7f263ad264a8>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120>
    {0.0} <music21.key.KeySignature of no sharps or flats>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.chord.Chord F2 A2 C3 F5>
    {4.0} <music21.chord.Chord E2 G2 C3 C5>
    {7.5} <music21.chord.Chord C5>
    {8.0} <music21.chord.Chord D2 F2 B-2 B-4>
    {9.0} <music21.chord.Chord D5>
    {10.0} <music21.chord.Chord F5>
    {12.0} <music21.chord.Chord C3 F3 A3 A4>
    {13.0} <music21.chord.Chord C5>
    {14.0} <music21.chord.Chord F5>
    {15.5} <music21.chord.Chord A4>
    {16.0} <music21.chord.Chord B-2 D3 G3 G4>
    {17.0} <music21.chord.Chord B-4>
    {18.0} <music21.chord.Chord D5>
    {19.0} <music21.chord.Chord G4>
    {20.0} <music21.chord.Chord A2 C3 F3 F4>
    {21.0} <music21.chord.Chord A4>
    {22.0} <music21.chord.Chord C5>
    {23.5} <music21.chord.Chord C5>
    {24.0} <music21.chord.Chord G2 B2 D3 F3 B4>
  

In [23]:
text_stream.show('midi')

### Encode2text with durations

In [99]:
stream = file2stream(transpose_file) # 1.
s_arr = stream2chordarr(stream, encode_duration=False) # 2.
# s_arr = stream2chordarr(stream, encode_duration=True) # 2.
seq = chordarr2seq(s_arr) # 3.
text = seq2str_duration(seq)

In [90]:
s_arr[:10] = 0

In [91]:
s_arr[50:80] = 0

In [96]:
s_arr[-20:] = 0

In [92]:
s_arr.sum(axis=(1,2))[-20:]

array([-10.,  -9., -10.,  -9., -10.,  -9., -10.,  -4.,  -8.,  -8.,  -8.,   0.,   0.,   0.,   0.,  -4.,  -8.,  -8.,
        -8.,   0.])

In [93]:
remove_chordarr_rests(s_arr).shape

Compressing rests: 30 -> 18


(225, 2, 127)

In [94]:
remove_chordarr_rests(s_arr).sum(axis=(1,2))[-20:]

Compressing rests: 30 -> 18


array([-10.,  -9., -10.,  -9., -10.,  -9., -10.,  -4.,  -8.,  -8.,  -8.,   0.,   0.,   0.,   0.,  -4.,  -8.,  -8.,
        -8.,   0.])

In [97]:
remove_chordarr_rests(trim_chordarr_rests(s_arr)).shape

Trimming rests. Start, end: 8 217 20
Compressing rests: 30 -> 18


(197, 2, 127)

In [98]:
trim_chordarr_rests(s_arr).shape

Trimming rests. Start, end: 8 217 20


(209, 2, 127)

In [63]:
s_arr.shape

(237, 2, 127)

In [29]:
text

'|| t0 nF o2 t16 i0 nA o2 t16 i0 nC o3 t16 i0 nF o5 t16 i0 || t15 nE o2 t16 i0 nG o2 t16 i0 nC o3 t16 i0 nC o5 t12 i0 || t13 nC o5 t2 i0 || t1 nD o2 t16 i0 nF o2 t16 i0 nB- o2 t16 i0 nB- o4 t4 i0 || t3 nD o5 t4 i0 || t3 nF o5 t4 i0 || t7 nC o3 t16 i0 nF o3 t16 i0 nA o3 t16 i0 nA o4 t4 i0 || t3 nC o5 t4 i0 || t3 nF o5 t4 i0 || t5 nA o4 t2 i0 || t1 nB- o2 t16 i0 nD o3 t16 i0 nG o3 t16 i0 nG o4 t4 i0 || t3 nB- o4 t4 i0 || t3 nD o5 t4 i0 || t3 nG o4 t4 i0 || t3 nA o2 t16 i0 nC o3 t16 i0 nF o3 t16 i0 nF o4 t4 i0 || t3 nA o4 t4 i0 || t3 nC o5 t4 i0 || t5 nC o5 t2 i0 || t1 nG o2 t16 i0 nB o2 t16 i0 nD o3 t16 i0 nF o3 t16 i0 nB o4 t2 i0 || t1 nC o5 t2 i0 || t1 nD o5 t2 i0 || t1 nE o5 t2 i0 || t1 nF o5 t2 i0 || t1 nG o5 t2 i0 || t1 nG# o5 t2 i0 || t1 nC o5 t2 i0 || t1 nG o2 t8 i0 nB o2 t8 i0 nD o3 t8 i0 nF o3 t8 i0 nB o4 t2 i0 || t1 nC o5 t2 i0 || t1 nD o5 t2 i0 || t1 nE o5 t2 i0 || t1 nG# o2 t8 i0 nB o2 t8 i0 nD o3 t8 i0 nF o5 t2 i0 || t1 nG o5 t2 i0 || t1 nG# o5 t2 i0 || t1 nC o5 t2 i0 || t1 

In [42]:
text_stream = str2stream(text)
text_stream.show('midi')

In [106]:
for n in list(text_stream.flat[:200]):
    if isinstance(n, (music21.note.Note, music21.chord.Chord)):
        print(n.offset, n.duration, n.duration.quarterLength, n)

0.0 <music21.duration.Duration 4.25> 4.25 <music21.chord.Chord E2>
4.25 <music21.duration.Duration 0.25> 0.25 <music21.chord.Chord G3 E4 B4 E5>
4.5 <music21.duration.Duration 0.5> 0.5 <music21.chord.Chord G3 A3 E4 B4 C5 E5 F#5>
5.0 <music21.duration.Duration 0.25> 0.25 <music21.chord.Chord G3 A3 E4 B4 C5 E5 F#5>
5.25 <music21.duration.Duration 0.5> 0.5 <music21.chord.Chord G3 E4 E5 F#5>
5.75 <music21.duration.Duration 0.25> 0.25 <music21.chord.Chord E4 B4 C5>
6.0 <music21.duration.Duration 0.5> 0.5 <music21.chord.Chord G3 E4 B4 E5 B5>
6.5 <music21.duration.Duration 0.25> 0.25 <music21.chord.Chord B3>
6.75 <music21.duration.Duration 2.75> 2.75 <music21.chord.Chord A3 C4 E4 C5 E5 C6>
9.5 <music21.duration.Duration 1.0> 1.0 <music21.chord.Chord B3 E-4 A4 E-5>
10.5 <music21.duration.Duration 0.75> 0.75 <music21.chord.Chord G3 B4>
11.25 <music21.duration.Duration 0.5> 0.5 <music21.chord.Chord G3 E4 B4 E5 F#5>
11.75 <music21.duration.Duration 0.25> 0.25 <music21.chord.Chord A3 C5>
12.0 <musi

### intermediate encoding

In [115]:
def stream2chordarr(s, note_range=127, sample_freq=12):
    "Converts music21.Stream to 1-hot numpy array"
    # assuming 4/4 time
    # note x instrument x pitch
    # FYI: midi middle C value=60
    maxTimeStep = int(s.duration.quarterLength * sample_freq)+1
    
    # (AS) TODO: need to order by instruments most played and filter out percussion or include the channel
    inst2idx = {inst.id:idx for idx,inst in enumerate(s.flat.getInstruments())}
    score_arr = np.zeros((maxTimeStep, len(inst2idx), note_range))

    notes=[]
    noteFilter=music21.stream.filters.ClassFilter('Note')
    chordFilter=music21.stream.filters.ClassFilter('Chord')
    
    def note_data(pitch, note):
        inst_id = note.activeSite.getInstrument().id
        iidx = inst2idx[inst_id]
        return (pitch.midi, round(note.offset*sample_freq), round(note.duration.quarterLength*sample_freq), iidx)

    for n in s.recurse().addFilter(noteFilter):
        notes.append(note_data(n.pitch, n))
        
    for c in s.recurse().addFilter(chordFilter):
        pitchesInChord=c.pitches
        for p in pitchesInChord:
            notes.append(note_data(p, c))

    for n in notes:
        if n is None: continue
        pitch,offset,duration,inst = n
        score_arr[offset, inst, pitch] = VALTSTART                 # Strike note
        score_arr[offset+1:offset+duration, inst, pitch] = VALTCONT      # Continue holding note
    return score_arr


In [116]:
# 3.
def chordarr2stream(arr, sample_freq=12):
    duration = music21.duration.Duration(1. / sample_freq)
    stream = music21.stream.Stream()
    for inst in range(arr.shape[1]):
        p = partarr2stream(arr[:,inst,:], duration, stream=music21.stream.Part())
        stream.append(p)
    return stream

# 3b.
def partarr2stream(part, duration, stream=None, inst=None):
    "convert instrument part to music21 chords"
    if stream is None: stream = music21.stream.Stream()
    stream.append(music21.instrument.Piano())
    stream.append(music21.meter.TimeSignature(TIMESIG))
    stream.append(music21.tempo.MetronomeMark(number=120))
    stream.append(music21.key.KeySignature(0))
    starts = part == 1
    durations = calc_note_durations(part)
    for tidx,t in enumerate(starts):
        note_idxs = t.nonzero()[0]
        if len(note_idxs) == 0: continue
        notes = []
        for nidx in note_idxs:
            note = music21.note.Note(nidx)
            tnext = durations[tidx+1,nidx] if tidx+1 < len(part) else 0
            note.duration = music21.duration.Duration((tnext+1)*duration.quarterLength)
            notes.append(note)
        chord = music21.chord.Chord(notes)
        stream.insert(tidx*duration.quarterLength, chord)
#         chord = music21.chord.Chord(notes, offset=tidx*duration.quarterLength)
#         stream.append(chord)
    return stream
    
# 3c.
def calc_note_durations(part):
    "calculate midi note durations from TCONT notes"
    cnotes = (part == VALTCONT).astype(int)
    for i in reversed(range(cnotes.shape[0]-1)):
        cnotes[i] += cnotes[i+1]*cnotes[i]
    return cnotes


In [ ]:
test_stream.insert()

In [82]:
duration = music21.duration.Duration(1. / 4)

In [84]:
duration.quarterLength

0.25

In [86]:
694 * duration.quarterLength

173.5

In [85]:
part.shape

(694, 127)

In [61]:
part = charr[:, 0, :]

In [69]:
cnotes = (part == VALTCONT).astype(int)

In [78]:
cnotes = np.random.randint(0,2,(5,5))

In [79]:
cnotes

array([[1, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1],
       [1, 0, 0, 1, 0]])

In [80]:
for i in reversed(range(cnotes.shape[0]-1)):
    cnotes[i] += cnotes[i+1]*cnotes[i]

In [81]:
cnotes

array([[1, 2, 1, 5, 0],
       [0, 1, 0, 4, 0],
       [0, 0, 0, 3, 2],
       [0, 0, 0, 2, 1],
       [1, 0, 0, 1, 0]])

In [ ]:
cnotes 

In [90]:
np.trim_zeros(calc_note_durations(charr[:, 0, :]).sum(axis=1)).reshape(16, -1)

array([[  16,   15,   16,   13, ...,  249,  237,  225,  213],
       [ 222,  210,  199,  189, ...,  455,  446,  435,  426],
       [ 419,  450,  445,  439, ...,  996,  973,  954,  935],
       [ 916,  901,  886,  871, ...,  908,  894,  881,  872],
       ...,
       [1354, 1320, 1286, 1256, ...,  931,  906,  881,  856],
       [ 838,  811,  785,  763, ...,  701,  687,  659,  638],
       [ 617,  630,  602,  598, ...,  250,  233,  218,  203],
       [ 201,  187,  196,  183, ...,    4,    4,    2,    1]])

In [117]:
charr = stream2chordarr(transposed_stream)

In [118]:
charr.shape

(2081, 2, 127)

In [119]:
test_stream = chordarr2stream(charr)

In [120]:
test_stream.show('midi')